In [49]:
!pip install lxml

# [Homework 3](https://github.com/Sapienza-University-Rome/ADM/tree/master/2024/Homework_3) - Michelin restaurants in Italy
![iStock-654454404-777x518](https://a.storyblok.com/f/125576/2448x1220/327bb24d32/hero_update_michelin.jpg/m/1224x0/filters:format(webp))

## 1. Data collection

For the data collection, we wrote the required function in a `data_collection.py` module. 

In [50]:
from data_collection import save_links, download_html_from_link_file, html_to_tsv

The following is the overview of the main functions for each step, together with the code to run. 

Every function has an optional `data_folder` argument wich server the purpose to set the working data directory. 
We tought this to be useful, for example to set the date of the data collection as the directory name. 
This is useful, as the Michelin list of restaurant is constantly updated. 

In [51]:
data_folder = 'DATA 24-11-09'
# date of last data collection, yy-mm-dd

---

### 1.1. Get the list of Michelin restaurants
   #### **Function**: `save_links`
   - **Description**: 
     Collects restaurant links from the Michelin Guide website starting from the provided `start_url`. The links are saved into a text file (`restaurant_links.txt`) within a specified data folder.
   - **Input**: 
     - `start_url`: URL of the Michelin Guide page to start scraping.
   - **Optional Input**: 
     - `file_name`: name of the output file; by default it is `restaurant_links.txt`.
     - `data_folder`: the folder where datas will be stored; by default it is `DATA`.
   - **Output**:
     - A text file containing restaurant links, one per line, saved in the `data_folder`.
   - **Key Features**:
     - Automatically detects the number of pages to scrape.
     - Skips scraping if the links file already exists.

In [52]:
start_url = "https://guide.michelin.com/en/it/restaurants"
save_links(start_url, data_folder = data_folder)

Links already collected.
There are 1982 link already collected


---

### 1.2. Crawl Michelin restaurant pages
   #### **Function**: `download_html_from_link_file`
   - **Description**: 
     Downloads the HTML from every URL in the input `file_name`, and saves them to a structured folder (`DATA/HTMLs/page_X`).
   - **Input (all optional)**:
     - `file_name`: name of the file with the links; by default it is `restaurant_links.txt`.
     - `data_folder`: the folder where datas will be stored; by default it is `DATA`.
   - **Output**:
     - Saves the HTML files in a structured folder `DATA/HTMLs/page_X`. 
   - **Key Features**:
     - Uses `ThreadPoolExecutor` to speed up the process
     - Skips existing HTML files

In [53]:
download_html_from_link_file(data_folder = data_folder)

Download HTMLs: 100%|██████████| 1982/1982 [00:00<00:00, 5154.26it/s]

All html files have been saved.


---

### 1.3 Parse downloaded pages

#### **Function**: `extract_info_from_html`
- **Description**:  
  Parses a restaurant's HTML page and extracts structured information such as name, address, cuisine type, price range, description, and services.
- **Input**:
  - `html`: The raw HTML content of a restaurant's page.
- **Output**:
  - A dictionary containing extracted fields.
- **Key Features**:
  - Handles missing data gracefully.
  - Handles addresses separated by commas.


#### **Function**: `html_to_tsv`
- **Description**:  
  Scans the `HTMLs` folder inside the `data_folder` for all the html files, then processes every file with `extract_info_from_html`.
- **Input (optional)**:
  - `data_folder`: The folder where data will be stored; by default it is `DATA`.
  - `max_workers`: the max number of concurrent HTML parsing tasks. 
- **Output**:
  - Saves the TSV files in the folder `DATA/TSVs`.
- **Key Features**:
     - Uses `ThreadPoolExecutor` to speed up the process. 
- **Advice**:
     - Fine-tune the `max_workers` parameter according to your CPU performance. As a rule of thumb, set `max_workers` to the number of CPU cores available. An estimated processing time of around 5 minutes is typical. 

In [54]:
html_to_tsv(data_folder=data_folder, max_workers=4)

Processing HTMLs: 100%|██████████| 1982/1982 [00:00<00:00, 30033.02it/s]

All files have been processed and saved.


For completeness, let us create the dataframe for our dataset, in order to handle it effectively.

#### **Function**: `create_combined_dataframe`
- **Description**:  
  This function reads all the `.tsv` files from a specified folder, loads them into individual pandas DataFrames, and then combines them into a single DataFrame. It is useful for aggregating data from multiple sources into one unified dataset for further analysis.

- **Input**:
  - `folder_path` (str): The path to the folder containing the `.tsv` files to be read.
  - `separator` (str): The delimiter used in the `.tsv` files. Typically, it's a tab (`\t`), but it could be adjusted if needed.
  
- **Output**:
  - Returns a pandas DataFrame containing all the combined data from the `.tsv` files in the specified folder.

- **Key Features**:
  - Utilizes `glob` to find all `.tsv` files in the provided folder.
  - Loads each file as a DataFrame using pandas `read_csv()` with the specified delimiter.
  - Concatenates all DataFrames into one, ignoring index to prevent duplication.
  - Efficient handling of large datasets through pandas' built-in functions.

By running this function, you'll have a consolidated view of all the restaurant data in a single DataFrame, ready for any further analysis or processing. The first few rows of the dataset are provided below.

In [55]:
from data_collection import create_combined_dataframe
df = create_combined_dataframe(data_folder+"\TSVs", separator='\t')
df.head()

,restaurantName,address,city,postalCode,country,priceRange,cuisineType,description,facilitiesServices,creditCards,phoneNumber,website
0,20Tre,via David Chiossone 20 r,Genoa,16123,Italy,€€,"Farm to table, Modern Cuisine",Situated in the heart of Genoa’s historic cent...,['Air conditioning'],"['amex', 'dinersclub', 'mastercard', 'visa']",+39 010 247 6191,https://www.ristorante20tregenova.it/
1,Il Ristorante Alain Ducasse Napoli,Via Cristoforo Colombo 45,Naples,80133,Italy,€€€€,"Creative, Mediterranean Cuisine","Alain Ducasse, one of the great names in conte...","['Air conditioning', 'Great view', 'Interestin...","['amex', 'dinersclub', 'discover', 'maestrocar...",+39 081 604 1580,https://theromeocollection.com/en/romeo-napoli...
2,Salvo Cacciatori,via Vieusseux 12,Oneglia,18100,Italy,€€€,"Ligurian, Contemporary",This restaurant has come a long way since 1906...,"['Air conditioning', 'Restaurant offering vege...","['amex', 'discover', 'jcb', 'maestrocard', 'ma...",+39 0183 293763,https://ristorantesalvocacciatori.it/
3,Terre Alte,"via Olmadella 11, località Balignano",Longiano,47020,Italy,€€€,Seafood,One of the best-known addresses in this region...,"['Air conditioning', 'Car park', 'Terrace']","['amex', 'mastercard', 'visa']",+39 0547 666138,https://ristoranteterrealte.com/
4,Tubladel,via Trebinger 22,Ortisei,39046,Italy,€€€,Regional Cuisine,Although this restaurant with wood-adorned din...,"['Car park', 'Terrace']","['mastercard', 'visa']",+39 0471 796879,https://www.tubladel.com/


---

## 2. Search Engine

### 2.0.0) Preprocessing the Text

In [56]:
!pip install --upgrade nltk

In [76]:
import importlib, data_collection
importlib.reload(data_collection)

<module 'data_collection' from 'c:\\Users\\marta\\OneDrive\\Documenti\\GitHub\\ADM-HW3\\data_collection.py'>

In [58]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\marta\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\marta\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [59]:
from data_collection import preprocess_text
df['processedDescription'] = preprocess_text(df['description'])
df.head()

,restaurantName,address,city,postalCode,country,priceRange,cuisineType,description,facilitiesServices,creditCards,phoneNumber,website,processedDescription
0,20Tre,via David Chiossone 20 r,Genoa,16123,Italy,€€,"Farm to table, Modern Cuisine",Situated in the heart of Genoa’s historic cent...,['Air conditioning'],"['amex', 'dinersclub', 'mastercard', 'visa']",+39 010 247 6191,https://www.ristorante20tregenova.it/,"[situat, heart, genoa, histor, centr, contempo..."
1,Il Ristorante Alain Ducasse Napoli,Via Cristoforo Colombo 45,Naples,80133,Italy,€€€€,"Creative, Mediterranean Cuisine","Alain Ducasse, one of the great names in conte...","['Air conditioning', 'Great view', 'Interestin...","['amex', 'dinersclub', 'discover', 'maestrocar...",+39 081 604 1580,https://theromeocollection.com/en/romeo-napoli...,"[alain, ducass, one, great, name, contemporari..."
2,Salvo Cacciatori,via Vieusseux 12,Oneglia,18100,Italy,€€€,"Ligurian, Contemporary",This restaurant has come a long way since 1906...,"['Air conditioning', 'Restaurant offering vege...","['amex', 'discover', 'jcb', 'maestrocard', 'ma...",+39 0183 293763,https://ristorantesalvocacciatori.it/,"[restaur, come, long, way, sinc, 1906, owner, ..."
3,Terre Alte,"via Olmadella 11, località Balignano",Longiano,47020,Italy,€€€,Seafood,One of the best-known addresses in this region...,"['Air conditioning', 'Car park', 'Terrace']","['amex', 'mastercard', 'visa']",+39 0547 666138,https://ristoranteterrealte.com/,"[one, best, known, address, region, fish, enth..."
4,Tubladel,via Trebinger 22,Ortisei,39046,Italy,€€€,Regional Cuisine,Although this restaurant with wood-adorned din...,"['Car park', 'Terrace']","['mastercard', 'visa']",+39 0471 796879,https://www.tubladel.com/,"[although, restaur, wood, adorn, dine, room, r..."


In [71]:
from data_collection import create_vocabulary
vocabulary_df = create_vocabulary(df['processedDescription'])
print(vocabulary_df.head().to_string(index=False))

 term_id      term
       0 construct
       1  district
       2      farm
       3      icon
       4      nemi


In [82]:
from data_collection import create_inverted_index
inverted_index = create_inverted_index(df['processedDescription'], vocabulary_df)

Loading inverted index from file.


In [83]:
from data_collection import execute_query
query = "modern seasonal cuisine"
documents_id = execute_query(query, inverted_index, vocabulary_df)
df[['restaurantName', 'address', 'description', 'website']].iloc[sorted(documents_id)]

,restaurantName,address,description,website
88,Ronchi Rò,località Cime di Dolegna 12,Ronchi Rò is an estate-cum-agriturismo surroun...,https://www.ronchiro.it
180,Razzo,via Andrea Doria 17/f,"A quiet restaurant with a relaxed, young and m...",https://vadoarazzo.it/
256,Flurin,Laubengasse 2,Flurin occupies an old medieval tower in Glore...,https://www.flurin.it
289,Quadri Bistrot,Via Solferino 48,"A modern bistro with a cocktail-bar, trendy de...",https://www.quadribistrot.it/
304,Materia | Spazio Cucina,via Teatro Massimo 29,The entrance to this restaurant is typical of ...,https://www.materiaspaziocucina.it/
456,Gallery Bistrot Contemporaneo,via Regina Margherita 3/b,"Modern, tasty and carefully curated cuisine, w...",NaN
501,Ca' Del Moro,località Erbin 31,Situated within the La Collina dei Ciliegi win...,https://www.cadelmoro.wine/it
561,[àbitat],via Henry Dunant 1,"A young, enthusiastic and professional couple ...",https://www.abitatproject.it
565,Babette,via Michelangelo 17,Situated just beyond the centre of Albenga in ...,https://www.ristorantebabette.net/
586,Cappuccini Cucina San Francesco,via Cappuccini 54,"Housed in the resort of the same name, this el...",https://www.cappuccini.it/
